In [1]:
import pandas as pd
from pandas import read_csv
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load all the blocks and concatenate them into one database
path = '/Users/mariabelenalberti/OneDrive - Universidad Torcuato Di Tella/Tesis/smart-meters-in-london/halfhourly_dataset' # use your path
all_files = glob.glob(path + "/*.csv")

lista = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    lista.append(df)

halfhourly_consumption = pd.concat(lista, axis=0, ignore_index=True)

/Users/mariabelenalberti/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(halfhourly_consumption)

167817021

In [4]:
halfhourly_consumption.head()

,LCLid,tstp,energy(kWh/hh)
0,MAC000027,2011-12-07 11:30:00.0000000,0.185
1,MAC000027,2011-12-07 12:00:00.0000000,0.155
2,MAC000027,2011-12-07 12:30:00.0000000,0.147
3,MAC000027,2011-12-07 13:00:00.0000000,0.164
4,MAC000027,2011-12-07 13:30:00.0000000,0.187


In [5]:
halfhourly_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167817021 entries, 0 to 167817020
Data columns (total 3 columns):
LCLid             object
tstp              object
energy(kWh/hh)    object
dtypes: object(3)
memory usage: 3.8+ GB


In [7]:
halfhourly_consumption.rename(columns={"energy(kWh/hh)": "energy"})
halfhourly_consumption['tstp']= pd.to_datetime(halfhourly_consumption['tstp'],format = '%Y-%m-%d')
halfhourly_consumption['year'] = pd.DatetimeIndex(halfhourly_consumption['tstp']).year

In [ ]:
halfhourly_consumption

In [8]:
# Filter for 2012 and 2013
halfhourly_consumption_12_13 = halfhourly_consumption[(halfhourly_consumption['year'] == 2012) | (halfhourly_consumption['year'] == 2013)]                     

In [9]:
len(halfhourly_consumption_12_13)

153392724

In [10]:
LCLid = halfhourly_consumption_12_13.LCLid

In [11]:
len(LCLid)

153392724

In [16]:
LCLid_distinct = LCLid.drop_duplicates().to_list()
LCLid_distinct.sort()

In [17]:
LCLid_distinct = pd.DataFrame(LCLid_distinct)

In [18]:
len(LCLid_distinct)

5566

In [20]:
LCLid_distinct.to_csv("LCLid_distinct.csv",index=False)

In [88]:
for household in LCLid_distinct:
    household_mac = halfhourly_consumption[halfhourly_consumption['LCLid'] == LCLid_distinct[i]]
    del household_mac['LCLid']
    del household_mac['year']
    df = pd.DataFrame(household_mac)
    df.to_csv("household"+"_"+str(household)+ ".csv",index=False)

KeyboardInterrupt: 